In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
plt.style.use('default')
#plt.style.use("seaborn-poster")
plt.style.use("seaborn-talk")
from combined_phot_funcs import *
from photometryQuantifications import *#removeErrantBlock1Assignments
%matplotlib qt

# create saline and cocaine photometry PhotRats objects.
- seshdicts are dictionaries of key = rat and values = dates within rat.

In [ ]:
seshDict = {"IM-1532":["12142022"],"IM-1533":["12142022"]}#{"IM-1434":["04132022","04192022"]}
salRats = PhotRats(seshDict)
salRats.directory_prefix="/Volumes/Tim/Photometry/"
salRats.load_dfs()
seshDict = {"IM-1532":["12132022"],"IM-1533":["12132022"]}#{"IM-1434":["04202022"]}
cokeRats = PhotRats(seshDict)
cokeRats.directory_prefix="/Volumes/Tim/Photometry/"
cokeRats.load_dfs()

## minor data processing for memory etc.

In [ ]:
cokeRats.df.loc[cokeRats.df.port.isnull(),"port"]=-100
salRats.df.loc[salRats.df.port.isnull(),"port"]=-100
cokeRats.df.loc[cokeRats.df.rwd.isnull(),'rwd'] = 0
salRats.df.loc[salRats.df.rwd.isnull(),'rwd'] = 0

cokeRats.df.loc[:,"session"] = cokeRats.df.loc[:,"session"].astype("int16")
cokeRats.df.loc[:,"block"] = cokeRats.df.loc[:,"block"].astype("int8")
cokeRats.df.loc[:,"port"] = cokeRats.df.loc[:,"port"].astype("int8")
cokeRats.df.loc[:,"rwd"] = cokeRats.df.loc[:,"rwd"].astype("int8")
salRats.df.loc[:,"session"] = salRats.df.loc[:,"session"].astype("int16")
salRats.df.loc[:,"block"] = salRats.df.loc[:,"block"].astype("int8")
salRats.df.loc[:,"port"] = salRats.df.loc[:,"port"].astype("int8")
salRats.df.loc[:,"rwd"] = salRats.df.loc[:,"rwd"].astype("int8")

removeErrantBlock1Assignments(salRats)
removeErrantBlock1Assignments(cokeRats)

# create columns indicating pre or post injection

In [ ]:
cokeRats.df.loc[:,"post-Inj"] = 0
cokeRats.df.loc[:,"pre-Inj"] = 0
salRats.df.loc[:,"post-Inj"] = 0
salRats.df.loc[:,"pre-Inj"] = 0

# for each rat and date in the cocaine object, input the time of injection (in minutes). This will fill in the dataframe columns, marking post-injection as 20min after, but this can be changed

In [ ]:
timeAfterInj = 15
sessionStartExcludeTime = 10 # this deserves more discussion

In [ ]:
rat = input("rat ID include IM-")
date = int(input("date"))
timeOfInj = int(input("time of injection? (min)"))
timeOfInj_sampleTime = timeOfInj*60*250
dat = cokeRats.df.loc[(cokeRats.df.rat==rat)&(cokeRats.df.date==date),:].copy()
startFill = dat.index.min()+timeOfInj_sampleTime+timeAfterInj*250*60
dat.loc[startFill:,"post-Inj"] = 1
dat.loc[sessionStartExcludeTime*250*60:dat.index.min()+timeOfInj_sampleTime,"pre-Inj"] = 1
cokeRats.df.loc[(cokeRats.df.rat==rat)&(cokeRats.df.date==date),:] = dat

# do the same for saline rats. edit to add variables for time inclusion/exclusion

In [ ]:
rat = input("rat ID include IM-")
date = int(input("date"))
timeOfInj = int(input("time of injection? (min)"))
timeOfInj_sampleTime = timeOfInj*60*250
dat = salRats.df.loc[(salRats.df.rat==rat)&(salRats.df.date==date),:].copy()
startFill = dat.index.min()+timeOfInj_sampleTime+20*250*60
dat.loc[startFill:,"post-Inj"] = 1
dat.loc[10*250*60:dat.index.min()+timeOfInj_sampleTime,"pre-Inj"] = 1
salRats.df.loc[(salRats.df.rat==rat)&(salRats.df.date==date),:] = dat

In [ ]:
create_triframe(cokeRats)
create_triframe(salRats)

In [ ]:
def get_log_pchoos_v_costNben(photrats,postInj=True):
    '''if postInj is true, only consider trials after injection,
    else, consider pre-injection time points.'''
    df = photrats.triframe.loc[photrats.triframe["post-Inj"==1].copy() if postInj\
        else photrats.triframe.loc[photrats.triframe["pre-Inj"==1].copy()
    df['ratcodes'] = df.rat.astype('category').cat.codes
    seshs=df.session.unique()
    for s in range(len(seshs)):
        sdf = df.loc[(df.session==seshs[s])].copy()
        rdf = pd.DataFrame({'pRwdDif':photrats.get_lr_dif_val(\
            sdf,'nom_rwd'),'ldif':photrats.get_lr_dif_val(sdf,'dist')})
        rdf['choose_L'] = sdf.lrchoice.values
        rdf['session']=s
        rdf['rat'] = sdf.ratcodes.values
        rdf['tri'] = sdf.tri.values
        rdf['block'] = sdf.block.values
        rdf.loc[:,"rt-1"] = np.nan
        sdf.reset_index(inplace=True)
        for p in range(3):
            lagInds = sdf.loc[sdf.port==p,:].index
            rdf.loc[lagInds,"rt-1"] = sdf.loc[lagInds,'rwd'].shift(1).values
        if s == 0:
            photrats.regdf = rdf
        else:
            photrats.regdf = photrats.regdf.append(rdf,ignore_index=True)
    photrats.regdf.loc[photrats.regdf.choose_L==2,'choose_L']=np.nan